In [1]:
import pandas as pd
from src.ConectorBD import *

In [2]:
import pandasql
from pandasql import sqldf

In [3]:
import koinenlp

In [4]:
# %pip list | grep cltk

In [5]:
# %pip install cltk==0.1.121 --force-reinstall

In [4]:
import nltk

In [5]:
import cltk

/home/davison/Software/biblia-semantica/.venv/lib/python3.13/site-packages/cltk/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution


In [8]:
# This is the import of the right part of the CLTK library
from cltk.corpus.utils.importer import CorpusImporter

In [9]:
# Let's get some Greek corpora, too

my_greek_downloader = CorpusImporter('greek')
my_greek_downloader.import_corpus('greek_models_cltk')
my_greek_downloader.list_corpora

['greek_software_tlgu',
 'greek_text_perseus',
 'phi7',
 'tlg',
 'greek_proper_names_cltk',
 'greek_models_cltk',
 'greek_treebank_perseus',
 'greek_treebank_gorman',
 'greek_lexica_perseus',
 'greek_training_set_sentence_cltk',
 'greek_word2vec_cltk',
 'greek_text_lacus_curtius',
 'greek_text_first1kgreek',
 'greek_text_tesserae']

In [10]:
# corpus_importer = CorpusImporter('hebrew')

In [11]:
# corpus_importer.list_corpora

In [12]:
# AT

In [6]:
column_names = ['book_nr', 'chapter_nr', 'verse_nr', 'part_of_speech', 'morphology', 'wd_sep_strongs']
lxx_source = pd.read_fwf('/home/davison/Software/biblia-semantica/lxxm_utf8_v4', colspecs='infer', 
                         header=None, names=column_names, widths=[2, 3, 3, 4, 9, 50])

In [14]:
lxx_source.shape

(475400, 6)

In [7]:
# Split the 'Address' column by comma and expand into new columns
df_split = lxx_source['wd_sep_strongs'].str.split(' ', expand=True)

# Assign names to the new columns (optional, if you know the number of splits)
df_split.columns = ['normalized_word', 'strongs']

lxx_source.drop(['wd_sep_strongs'], axis=1, inplace=True)

# Concatenate the new columns back to the original DataFrame
lxx_source = pd.concat([lxx_source, df_split], axis=1)

In [60]:
from cltk.stem.lemma import LemmaReplacer
# from cltk.corpus.utils.formatter import cltk_normalize

In [ ]:
# sentence = cltk_normalize(sentence)  # can help when using certain texts

In [19]:
lemmatizer = LemmaReplacer('greek')

In [57]:
lemmatized_sentence = lemmatizer.lemmatize(lxx_exp)

In [58]:
lemmatized_sentence 

['ματαιοτης']

In [45]:
lemmatizer.lemmatize(lxx_source['normalized_word'][0])[0]

'εν'

In [46]:
new_col_values = [] 

# # criando coluna com texto 'normalizado'
for i, row in lxx_source.iterrows():
    new_col_values.append(lemmatizer.lemmatize(row['normalized_word'].strip())[0])

lxx_source['lemma'] = new_col_values

In [58]:
lxx_source.query("was_lemmatized == True")

,book_nr,chapter_nr,verse_nr,part_of_speech,morphology,normalized_word,strongs,lemma,was_lemmatized
27,1,1,2,RA,----GSN-,του,3588,τις,True
52,1,1,4,RA,----GSN-,του,3588,τις,True
57,1,1,4,RA,----GSN-,του,3588,τις,True
87,1,1,6,RA,----GSN-,του,3588,τις,True
112,1,1,7,RA,----GSN-,του,3588,τις,True
...,...,...,...,...,...,...,...,...,...
475254,39,3,18,RA,----GSM-,του,3588,τις,True
475258,39,3,18,RA,----GSM-,του,3588,τις,True
475309,39,4,2,RP,----GS--,μου,3450,ἐγώ,True
475385,39,4,4,RA,----GSM-,του,3588,τις,True


In [47]:
import numpy as np

lxx_source['was_lemmatized'] = np.where(lxx_source['lemma'] != lxx_source['normalized_word'], True, False)

In [53]:
lemmatized_values = lxx_source.query("was_lemmatized == True").values

In [57]:
lemmatized_values

array([[1, 1, 2, ..., '3588', 'τις', True],
       [1, 1, 4, ..., '3588', 'τις', True],
       [1, 1, 4, ..., '3588', 'τις', True],
       ...,
       [39, 4, 2, ..., '3450', 'ἐγώ', True],
       [39, 4, 4, ..., '3588', 'τις', True],
       [39, 4, 4, ..., '3450', 'ἐγώ', True]],
      shape=(25866, 9), dtype=object)

In [ ]:

# lxx_source.drop(['lemma'], axis=1, inplace=True)

In [16]:
# del lxx

In [18]:
lxx_source["book_nr"].nunique()

39

In [8]:
lxx_agrupada_versos = sqldf("""
                            select 
                                book_nr
                                , chapter_nr
                                , verse_nr
                                , group_concat(normalized_word, ' ') as txt
                            FROM 
                                lxx_source
                            group by
                                book_nr
                                , chapter_nr
                                , verse_nr            
                            """)

In [20]:
# lxx_agrupada_versos.head()

In [21]:
# NT

In [9]:
sbl_bd = '/home/davison/Software/biblia-semantica/db/sbl.db'

In [23]:
# sql_sbl = "SELECT nr_sq_livro, chapter, verse, word, word_nr FROM words"

In [10]:
sql_sbl = "SELECT * FROM words"

In [25]:
# sql_sbl = "SELECT * FROM bible_fts"

In [11]:
gnt_df = pd.read_sql_query(sql_sbl, ConectorBD(sbl_bd).con)

In [27]:
gnt_df.head()

,book,chapter,verse,pos,morph,lemma,strongs,word,punct,word_nr,nr_sq_livro
0,Matt,1,1,N-,----NSF-,βίβλος,00976,Βίβλος,,1,40
1,Matt,1,1,N-,----GSF-,γένεσις,01078,γενέσεως,,2,40
2,Matt,1,1,N-,----GSM-,Ἰησοῦς,02424,Ἰησοῦ,,3,40
3,Matt,1,1,N-,----GSM-,Χριστός,05547,χριστοῦ,,4,40
4,Matt,1,1,N-,----GSM-,υἱός,05207,υἱοῦ,,5,40


In [28]:
# # del gnt_normal_txt
# gnt_df = sqldf("select * from gnt_df")

In [29]:
# new_col_values = [] 

# # sem normalização (só tira as quebras)
# for i, row in gnt_df.iterrows():
#     new_col_values.append(row['content'].strip())
    
# gnt_df['content'] = new_col_values

In [30]:
# new_col_values = [] 

# # criando coluna com texto 'normalizado'
# for i, row in gnt_df.iterrows():
#     new_col_values.append(koinenlp.normalize(row['content'].strip()))

# gnt_df['normalized_text'] = new_col_values

In [31]:
# gnt_df.head()

In [32]:
# gnt_df.shape

In [33]:
# gnt_df['chapter'] = pd.to_numeric(gnt_df['chapter'], errors='coerce')
# gnt_df['verse'] = pd.to_numeric(gnt_df['verse'], errors='coerce')

In [34]:
print(gnt_df.dtypes)

book           object
chapter         int64
verse           int64
pos            object
morph          object
lemma          object
strongs        object
word           object
punct          object
word_nr         int64
nr_sq_livro     int64
dtype: object


In [35]:
gnt_df["book"].unique()

array(['Matt', 'Mark', 'Luke', 'John', 'Acts', 'Rom', '1Cor', '2Cor',
       'Gal', 'Eph', 'Phil', 'Col', '1Thess', '2Thess', '1Tim', '2Tim',
       'Titus', 'Phlm', 'Heb', 'Jas', '1Pet', '2Pet', '1John', '2John',
       '3John', 'Jude', 'Rev'], dtype=object)

In [36]:
# a partir daqui busca expressão GNT na LXX
# GNT só precisa normalize, pq já tem a coluna lemma


# nesse caso, vida e eterna é a mesma morfologia das palavras 
# que aparece na oração sacerdotal de João 17.3 e Daniel 12.2 (pelo lemma não acha)

gnt_df.query("book == 'John' and chapter == 17 and verse == 2")["word"].values[13:15]

array(['ζωὴν', 'αἰώνιον'], dtype=object)

In [39]:
gnt_vs_wds = " ".join(gnt_df.query("book == 'Rom' and chapter == 8 and verse == 20")['word'].values)

In [42]:
gnt_vs_wds

'τῇ γὰρ ματαιότητι ἡ κτίσις ὑπετάγη οὐχ ἑκοῦσα ἀλλὰ διὰ τὸν ὑποτάξαντα ἐφ’ ἑλπίδι'

In [40]:
lemmatized_sentence = lemmatizer.lemmatize(gnt_vs_wds)

In [41]:
lemmatized_sentence

['ὁ',
 'γὰρ',
 'ματαιότης',
 'ὁ',
 'κτίσις',
 'ὑποτάσσω',
 'οὐ',
 'ἑκών',
 'ἀλλὰ',
 'διὰ',
 'τὸν',
 'ὑποτάσσω',
 'ἐπί',
 'ἑλπίδι']

In [31]:
gnt_vs = [koinenlp.normalize(wd) for wd in 
                  gnt_df.query("book == 'John' and chapter == 8 and verse == 34")["word"].values]

In [36]:
gnt_vs

['απεκριθη',
 'αυτοισ',
 'ο',
 'ιησουσ',
 'αμην',
 'αμην',
 'λεγω',
 'υμιν',
 'οτι',
 'πασ',
 'ο',
 'ποιων',
 'την',
 'αμαρτιαν',
 'δουλοσ',
 'εστιν',
 'τησ',
 'αμαρτιασ']

In [32]:
gnt_wd1, gnt_wd2 = gnt_vs[11], gnt_vs[13]

In [35]:
print(gnt_wd1, gnt_wd2)

ποιων αμαρτιαν


In [33]:
# busca lista de palavras em qualquer posição na lxx (normalizadas) 

ocorrencias_exp_lxx = sqldf("""
                            select * 
                            from lxx_agrupada_versos 
                            where 
                                txt like '%{}%' 
                            AND txt like '%{}%'
                            """.format(gnt_wd1, gnt_wd2))

In [34]:
ocorrencias_exp_lxx.head()

,book_nr,chapter_nr,verse_nr,txt


In [47]:
# a partir daqui buscar palavra/expressão da LXX no GNT
# LXX só precisa lemmatizer 



In [48]:
# eu vivo eternamente
lxx_exp = " ".join(lxx_source.query("book_nr == 5 and chapter_nr == 32 and verse_nr == 40")["normalized_word"][15:20])

In [60]:
# vida eterna 
lxx_exp = " ".join(lxx_source.query("book_nr == 27 and chapter_nr == 12 and verse_nr == 2")["normalized_word"][10:12])

In [50]:
# vaidade (última palavra de Eclesiastes 1.2)
lxx_exp = lxx_source.query("book_nr == 21 and chapter_nr == 1 and verse_nr == 2")["normalized_word"].values[-1]

In [51]:
lxx_source.query("book_nr == 21 and chapter_nr == 1 and verse_nr == 2")["normalized_word"].values[-1]

'ματαιοτης'

In [52]:
lxx_exp

'ματαιοτης'

In [61]:
lst_palavras_exp_lxx = lxx_exp.split(" ")

In [62]:
lxx_wd1, lxx_wd2 = lst_palavras_exp_lxx[0], lst_palavras_exp_lxx[1]

In [59]:
lxx_wd1 = lemmatized_sentence[0]

In [ ]:
# qualquer ocorrência de lxx_wd1 no gnt (normalizado)
# ocorrencias_exp_gnt = sqldf("""
#                             select * 
#                             from gnt_df  
#                             where 
#                                 word like '%{}%' 
#                             """.format(lxx_wd1))

In [ ]:
# busca lista de palavras em qualquer posição no GNT (normalizadas)
ocorrencias_exp_gnt = sqldf("""
                            select * 
                            from gnt_df 
                            where 
                                normalized_text like '%{}%' 
                            OR  normalized_text like '%{}%'
                            """.format(lxx_wd1, lxx_wd2))

In [64]:
lxx_agrupada_versos.head()

,book_nr,chapter_nr,verse_nr,txt
0,1,1,1,εν αρχη εποιησεν ο θεος τον ουρανον και την γην
1,1,1,2,η δε γη ην αορατος και ακατασκευαστος και σκοτ...
2,1,1,3,και ειπεν ο θεος γενηθητω φως και εγενετο φως
3,1,1,4,και ειδεν ο θεος το φως οτι καλον και διεχωρισ...
4,1,1,5,και εκαλεσεν ο θεος το φως ημεραν και το σκοτο...


In [65]:
ocorrencias_exp_gnt = sqldf("""
                            select * 
                            from lxx_agrupada_versos 
                            where 
                                txt like '%{}%' 
                            AND  txt like '%{}%'
                            """.format(lxx_wd1, lxx_wd2))

In [66]:
len(ocorrencias_exp_gnt)

1

In [67]:
ocorrencias_exp_gnt.head()

,book_nr,chapter_nr,verse_nr,txt
0,27,12,2,και πολλοι των καθευδοντων εν γης χωματι εξεγε...


In [ ]:
# gnt_df_gk_wd = pd.read_sql_query("SELECT * FROM words WHERE strongs like '3776'", ConectorBD(gnt).con)

In [ ]:
# gnt_grouped_df = pd.read_sql_query("""
#                            SELECT
#                                 nr_sq_livro
#                                 , chapter
#                                 , verse
#                                 , group_concat(word, ' ') as words
#                                 , group_concat(lemma, ' ') as lemmas
#                                 , group_concat(strongs, ' ') as strongs
#                             FROM 
#                                 words
#                             group by
#                                 nr_sq_livro
#                                 , chapter
#                                 , verse""", ConectorBD(gnt).con)

In [ ]:
gnt_df_gk_wd.head()

In [ ]:
# palavra_grega = koinenlp.normalize(strongs_grk_df['lemma'][1])
# import re
# palavra_grega = koinenlp.normalize(gnt_df.loc[(gnt_df['nr_sq_livro'] == 45) & (gnt_df['chapter'] == 10) & (gnt_df['verse'] == 9)].values[2][5])
# print(palavra_grega)
# df_query = lxx_df["verse_text"].str.contains(palavra_grega, flags=re.IGNORECASE, regex=True)
# df_query.value_counts()


ομολογεω


verse_text
False    30607
Name: count, dtype: int64